In [1]:
import os
import requests
import regex as re
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

### Download the water board's web page

In [3]:
url = "https://www.waterboards.ca.gov/drought/delta/"

In [4]:
page = requests.get(url, verify=False)

In [5]:
soup = BeautifulSoup(page.content, "html.parser")

In [6]:
content = soup.find("div", {"id": "DWCSL-accordion"})

### Parse curtailment lists

In [7]:
y2022 = content.find("div", { "id": "collapse-2022"}).find("ul")

In [8]:
list_2022 = y2022.find_all("li")

In [9]:
y2021 = content.find("div", { "id": "collapse-2021"}).find("ul")

In [10]:
list_2021 = y2021.find_all("li")

Concatenate

In [11]:
list_items = list_2022 + list_2021

In [19]:
dict_list = []
for li in list_items:
    text = li.text
    date = re.split("– | -", text)[0]
    url = li.find_all("a", href=True)[1]["href"]
    d = dict(
        text = li.text,
        date = re.split("– | -", text)[0],
        url = li.find_all("a", href=True)[1]["href"],
    )
    dict_list.append(d)

In [23]:
len(dict_list)

49

### Download each Excel file

In [21]:
path = '../data/raw/curtailments'

In [57]:
for item in dict_list:
    try:
        response = requests.get('https://www.waterboards.ca.gov/drought/delta/' + item["url"])
        datestr = item["date"].strip()
        dateslug = datetime.strptime(datestr, '%B %d, %Y').strftime('%Y-%m-%d')
        open(os.path.join(path, f'{dateslug}.xlsx'), 'wb').write(response.content)
    except:
        print("fail: " + item["date"])